In [1]:
import os
import sys

# Error message
def err(string):
    print(string)
    input("Press return to exit")
    sys.exit()

In [3]:
# Pivot sequence for LU and Gauss
def swapRows(v,i,j):
    if len(v.shape) == 1:
        v[i],v[j] = v[j],v[i]
    else:
        v[[i,j],:] = v[[j,i],:]

In [4]:
import numpy as np

def gaussPivot(a, b, tol=1.0e-12):
    n = len(b)
    
    # Set up scale factors
    s = np.zeros(n)
    for i in range (n):
        s[i] = max(np.abs(a[i,:]))
        
    for k in range(0, n-1):
        
        
        # Row interchange, if needed
        p = np.argmax(np.abs(a[k:n, k])/s[k:n]) + k
        if abs(a[p,k]) < tol: err("Matrix is singular")
        if p != k:
            swapRows(b, k, p)
            swapRows(s, k, p)
            swapRows(a, k, p)

        
        # Elimination
        for i in range(k+1, n):
            if a[i, k] != 0.0:
                lam = a[i, k]/a[k, k]
                a[i, k+1:n] = a[i, k+1:n] - lam*a[k, k+1:n]
                b[i] = b[i] - lam*b[k]
    if abs(a[n-1, n-1]) < tol: err("Matrix is singular")
    
    
    # Back substitution
    b[n-1] = b[n-1]/a[n-1, n-1]
    for k in range(n-2, -1, -1):
        b[k] = (b[k] - np.dot(a[k, k+1:n], b[k+1:n]))/a[k, k]
    
    
    return b


In [7]:
import numpy as np

def swapRows(v,i,j):
    if len(v.shape) == 1:
        v[i],v[j] = v[j],v[i]
    else:
        v[[i,j],:] = v[[j,i],:]
        
def LUdecomp(a, tol=1e-9):
    """LU decomposition of matrix a using scaled row pivoting."""
    n = len(a)
    #Set up scale factors
    seq = np.array(range(n))
    s = np.zeros(n)


    for i in range(n):
        s[i] = max(abs(a[i, :]))


    for k in range(0, n - 1):
        #Row interchange, if needed
        p = np.argmax(np.abs(a[k:n, k]) / s[k:n]) + k
        if abs(a[p, k]) < tol:
            err('Matrix is singular')
        if p != k:
            swapRows(s, k, p)
            swapRows(a, k, p)
            swapRows(seq, k, p)


        #Elimination
        for i in range(k + 1, n):
            if a[i, k] != 0:
                lam = a[i, k] / a[k, k]
                a[i, k + 1:n] = a[i, k + 1:n] - lam * a[k, k + 1:n]
                a[i, k] = lam
    return a, seq


def LUsolve(a, b, seq):
    """Solves [L][U]{x} = {b}, where the matrix [a] and the permutation vector {seq} are returned from LUdecomp."""
    n = len(a)
    x = b.copy()


    #Rearrange constant vector; store result in x
    for i in range(n):
        x[i] = b[seq[i]]


    #Solution
    for k in range(1, n):
        x[k] = x[k] - np.dot(a[k, 0:k], x[0:k])
    x[n - 1] = x[n - 1] / a[n - 1, n - 1]
    for k in range(n - 2, -1, -1):
        x[k] = (x[k] - np.dot(a[k, k + 1:n], x[k + 1:n])) / a[k, k]
    return x

def matInv(a):
    n = len(a[0])
    aInv = np.identity(n)
    a, seq = LUdecomp(a)  # Retrieve a, seq from LU decomposition
    for i in range(n):
        aInv[:,i] = LUsolve(a, aInv[:,i], seq)
    
    return aInv



In [9]:
import numpy as np

def CramersRule(a, b):

    # Get dimensions of array
    dimensions = np.shape(a)
    rows, columns = dimensions
    
    # Calculate determinant of original matrix and initialize lists
    f = np.linalg.det(a)
    var = []
    oglist = []
    
    # Replacing each column with b values
    for k in range(columns):

        for i in range(rows):
            oglist.append(a[i, k]) # Storing original values to substitute back in
            a[i, k] = b[i]
        
        # Determinant of new matrix
        l = np.linalg.det(a)
        var.append(l/f) # Store det(Ai)/det(A)

        # Put original values back in matrix
        for i in range(rows):
            a[i, k] = oglist[i]

        oglist.clear()
    
    return var

In [11]:
import numpy as np
import MyFunc
import time

def matrix_solver(matrix, rightside):
    
    # Timing and execution of gaussPivot function
    starttime_Gauss = time.perf_counter()
    b = MyFunc.gaussPivot(np.copy(matrix), np.copy(rightside))
    endtime_Gauss = time.perf_counter()

    # Accuracy of gaussPivot
    acc_Gauss = MyFunc.accuracy(np.copy(matrix), np.copy(rightside), b)

    # Get duration time for Gauss
    duration_Gauss = endtime_Gauss - starttime_Gauss

    # Timing and execution of LUdecomp function
    starttime_LUdecomp = time.perf_counter()
    d, seq = MyFunc.LUdecomp(np.copy(matrix))
    endtime_LUdecomp = time.perf_counter()

    
    duration_LUdecomp = endtime_LUdecomp - starttime_LUdecomp

    # Timing and execution of LUsolve function
    starttime_LUsolve = time.perf_counter()
    x = MyFunc.LUsolve(np.copy(d), np.copy(rightside), seq)
    endtime_LUsolve = time.perf_counter()

    duration_LUsolve = endtime_LUsolve - starttime_LUsolve

    # Add LUdecomp and LUsolve together to get total duration
    duration_LU = duration_LUsolve + duration_LUdecomp

    # Accuracy of LUPivot
    acc_LU = MyFunc.accuracy(np.copy(matrix), np.copy(rightside), x)

    # Timing and execution of Cramer function
    starttime_Cramer = time.perf_counter()
    var = MyFunc.CramersRule(matrix, rightside)
    endtime_Cramer = time.perf_counter()

    # Total duration time of Cramer function
    duration_Cramer = endtime_Cramer - starttime_Cramer

    # Reformat list var as an array
    p = np.zeros((len(var),1))

    for l in range(len(var)):
        p[l] = var[l]
    
    # Accuracy of Cramer function
    acc_Cramer = MyFunc.accuracy(np.copy(matrix), np.copy(rightside), p)

    return b, x, p, acc_Gauss, acc_LU, acc_Cramer, duration_Gauss, duration_LU, duration_Cramer

In [12]:
import numpy as np 

def accuracy(a, b, x):
    """acc = accuracy(a,b,x)
    Calculates the accuracy of your solution to a system of equations"""
    return np.linalg.norm(np.dot(a,x)-b)

In [15]:
def gaussSeidel(iterEqs, x, tol = 1.0e-9):
    omega = 1.0
    k = 10
    p = 1
    for i in range(1, 501):
        xOld = x.copy()
        x = iterEqs(x,omega)
        dx = np.sqrt(np.dot(x-xOld, x-xOld))
        if dx < tol: return x, i, omega
        # Compute relaxation factor after k + p iterations
        if i == k: dx1 = dx
        if i == k + p:
            dx2 = dx
            omega = 2.0/(1.0 + np.sqrt(1.0 - (dx2/dx1)**(1.0/p)))
    print("Gauss_Siedel failed to converge")

In [17]:
import math
import numpy as np

def conjGrad(Av,x,b, tol = 1.0e-9):
    n = len(b)
    r = b - Av(x)
    s = r.copy()
    for i in range(n):
        u = Av(s)
        alpha = np.dot(s,r)/np.dot(s,u)
        x = x + alpha*s
        r = b - Av(x)
        if(np.sqrt(np.dot(r,r))) < tol:
            break
        else:
            beta = -np.dot(r,u)/np.dot(s,u)
            s = r + beta*s
    return x, i

In [19]:
def Ax_Prob16(v):
    
    Ax = np.zeros(6)
    Ax[0] = 3.0*v[0] - 2.0*v[1] + v[2] + v[5]
    Ax[1] = -2.0*v[0] + 4.0*v[1] - 2.0*v[2] + v[3]
    Ax[2] = v[0] - 2.0*v[1] + 4.0*v[2] - 2.0*v[3] + v[4]
    Ax[3] = v[1] - 2.0*v[2] + 4.0*v[3] - 2.0*v[4] + v[5]
    Ax[4] = v[2] + -2.0*v[3] + 4.0*v[4] - 2.0*v[5]
    Ax[5] = v[0] + v[3] - 2.0*v[4] + 3.0*v[5]
    
    return Ax

In [21]:
def iterEqs_Prob16(x, omega):
    
    x[0] = omega/3.0*(10 - (-2.0*x[1] + x[2] + x[5])) + (1 - omega)*x[0]
    x[1] = omega/4.0*(-8 - (-2.0*x[0] - 2.0*x[2] + x[3])) + (1 - omega)*x[1]
    x[2] = omega/4.0*(10 - (x[0] - 2.0*x[1] - 2.0*x[3] + x[4])) + (1 - omega)*x[2]
    x[3] = omega/4.0*(10 - (x[1] - 2.0*x[2] - 2.0*x[4] + x[5])) + (1 - omega)*x[3]
    x[4] = omega/4.0*(-8 - (x[2] + -2.0*x[3] - 2.0*x[5])) + (1 - omega)*x[4]
    x[5] = omega/3.0*(10 - (x[0] + x[3] - 2.0*x[4])) + (1 - omega)*x[5]
    
    return x

In [23]:
def Ax_Prob18(v):
    
    n = len(v)
    Ax = np.zeros(n)
    Ax[0] = 4.0*v[0] - v[1] + v[n-1]
    for i in range(1, n-1):
        Ax[i] = -v[i-1] + 4.0*v[i] - v[i+1]
    Ax[n-1] = -v[n-2] + 4.0*v[n-1] + v[0]
    
    return Ax

In [25]:
def iterEqs_Prob17(x, omega):
    
    n = len(x)
    x[0] = omega*(x[1] - x[n-1])/4.0 + (1.0 - omega)*x[0]
    for i in range(1, n-1):
        x[i] = omega*(x[i-1] + x[i+1])/4.0 + (1.0 - omega)*x[i]
    x[n-1] = omega*(100.0 - x[0] + x[n-2])/4.0 + (1.0 - omega)*x[n-1]
    
    return x

In [27]:
def evalPoly(a,xData,x):
    n = len(xData) - 1 # Degree of polynomial
    p = a[n]
    for k in range(1,n+1):
        p = a[n-k] + (x -xData[n-k])*p
    return p

In [29]:
def coeffts(xData,yData):
    m = len(xData) # Number of data points
    a = yData.copy()
    for k in range(1,m):
        a[k:m] = (a[k:m] - a[k-1])/(xData[k:m] - xData[k-1])
    return a

In [31]:
import numpy as np

def rational(xData,yData,x):
    m = len(xData)
    r = yData.copy()
    rOld = np.zeros(m)
    for k in range(m-1):
        for i in range(m-k-1):
            if abs(x - xData[i+k+1]) < 1.0e-9:
                return yData[i+k+1]
            else:
                c1 = r[i+1] - r[i]
                c2 = r[i+1] - rOld[i+1]
                c3 = (x - xData[i])/(x - xData[i+k+1])
                r[i] = r[i+1] + c1/(c3*(1.0 - c1/c2) - 1.0)
                rOld[i+1] = r[i+1]
    return r[0]

In [33]:
import numpy as np

def LUdecomp3(c,d,e):
    n = len(d)
    for k in range(1,n):
        lam = c[k-1]/d[k-1]
        d[k] = d[k] - lam*e[k-1]
        c[k-1] = lam
    return c,d,e

def LUsolve3(c,d,e,b):
    n = len(d)
    for k in range(1,n):
        b[k] = b[k] - c[k-1]*b[k-1]
    b[n-1] = b[n-1]/d[n-1]
    for k in range(n-2,-1,-1):
        b[k] = (b[k] - e[k]*b[k+1])/d[k]
    return b

In [35]:
import numpy as np

def curvatures(xData,yData):
    n = len(xData) - 1
    c = np.zeros(n)
    d = np.ones(n+1)
    e = np.zeros(n)
    k = np.zeros(n+1)
    c[0:n-1] = xData[0:n-1] - xData[1:n]
    d[1:n] = 2.0*(xData[0:n-1] - xData[2:n+1])
    e[1:n] = xData[1:n] - xData[2:n+1]
    k[1:n] =6.0*(yData[0:n-1] - yData[1:n])/(xData[0:n-1] - xData[1:n]) - 6.0*(yData[1:n] - yData[2:n+1])/(xData[1:n] - xData[2:n+1])
    LUdecomp3(c,d,e)
    LUsolve3(c,d,e,k)
    return k

def evalSpline(xData,yData,k,x):
    
    def findSegment(xData,x):
        iLeft = 0
        iRight = len(xData)- 1
        while 1:
            if (iRight-iLeft) <= 1: return iLeft
            i =int((iLeft + iRight)/2)
            if x < xData[i]: iRight = i
            else: iLeft = i
    i = findSegment(xData,x)
    h = xData[i] - xData[i+1]
    y = ((x - xData[i+1])**3/h - (x - xData[i+1])*h)*k[i]/6.0 - ((x - xData[i])**3/h - (x - xData[i])*h)*k[i+1]/6.0 + (yData[i]*(x - xData[i+1]) - yData[i+1]*(x - xData[i]))/h
    
    return y

In [37]:
import numpy as np
import math

def polyFit(xData,yData,m):
    a = np.zeros((m+1,m+1))
    b = np.zeros(m+1)
    s = np.zeros(2*m+1)
    for i in range(len(xData)):
        temp = yData[i]
        for j in range(m+1):
            b[j] = b[j] + temp
            temp = temp*xData[i]
        temp = 1.0
        for j in range(2*m+1):
            s[j] = s[j] + temp
            temp = temp*xData[i]
    for i in range(m+1):
        for j in range(m+1):
            a[i,j] = s[i+j]
    return gaussPivot(a,b)

def stdDev(c,xData,yData):
    
    def evalPoly(c,x):
        m = len(c) - 1
        p = c[m]
        for j in range(m):
            p = p*x + c[m-j-1]
        return p
    
    n = len(xData) - 1
    m = len(c) - 1
    sigma = 0.0
    for i in range(n+1):
        p = evalPoly(c,xData[i])
        sigma = sigma + (yData[i] - p)**2
    sigma = math.sqrt(sigma/(n - m))
    return sigma

In [39]:
import numpy as np
import matplotlib.pyplot as plt

def plotPoly(xData,yData,coeff,xlab='x',ylab='y'):
    m = len(coeff)
    x1 = min(xData)
    x2 = max(xData)
    dx = (x2 - x1)/20.0
    x = np.arange(x1,x2 + dx/10.0,dx)
    y = np.zeros((len(x)))*1.0
    for i in range(m):
        y = y + coeff[i]*x**i
    plt.plot(xData,yData,'o',x,y,'-')
    plt.xlabel(xlab); plt.ylabel(ylab)
    plt.grid (True)
    plt.show()

In [41]:
from numpy import sign

def rootsearch(f,a,b,dx):
    x1 = a; f1 = f(a)
    x2 = a + dx; f2 = f(x2)
    while sign(f1) == sign(f2):
        if x1 >= b: return None,None
        x1 = x2; f1 = f2
        x2 = x1 + dx; f2 = f(x2)
    else:
        return x1,x2

In [43]:
def bisection(f,x1,x2,switch=1,tol=1.0e-9):

    f1 = f(x1)
    if f1 == 0.0: return x1
    f2 = f(x2)
    if f2 == 0.0: return x2
    if sign(f1) == sign(f2):
        err('Root is not bracketed')
    n = int(math.ceil(math.log(abs(x2 - x1)/tol)/math.log(2.0)))

    for i in range(n):
        x3 = 0.5*(x1 + x2); f3 = f(x3)
        if (switch == 1) and (abs(f3) > abs(f1)) and (abs(f3) > abs(f2)):
            return None

        if f3 == 0.0: return x3
        if sign(f2)!= sign(f3): x1 = x3; f1 = f3
        else: x2 = x3; f2 = f3
    return (x1 + x2)/2.0

In [45]:
def newtonRaphson(f,df,a,b,tol=1.0e-9):
    from numpy import sign
    fa = f(a)
    if fa == 0.0: return a
    fb = f(b)
    if fb == 0.0: return b
    if sign(fa) == sign(fb): err('Root is not bracketed')
    x = 0.5*(a + b)
    for i in range(30):
        fx = f(x)
        if fx == 0.0: return x
        # Tighten the brackets on the root
        if sign(fa) != sign(fx): b = x
        else: a = x

        # Try a Newton-Raphson step
        dfx = df(x)
        # If division by zero, push x out of bounds
        try: dx = -fx/dfx
        except ZeroDivisionError: dx = b - a
        x = x + dx
        # If the result is outside the brackets, use bisection
        if (b - x)*(x - a) < 0.0:
            dx = 0.5*(b - a)
            x = a + dx
        # Check for convergence
        if abs(dx) < tol*max(abs(b),1.0): return x
    print('Too many iterations in Newton-Raphson')

In [47]:
def newtonRaphson2(f,x,tol=1.0e-9):
    
    def jacobian(f,x):
        h = 1.0e-4
        n = len(x)
        jac = np.zeros((n,n))
        f0 = f(x)
        for i in range(n):
            temp = x[i]
            x[i] = temp + h
            f1 = f(x)
            x[i] = temp
            jac[:,i] = (f1 - f0)/h
        return jac, f0
    
    for i in range(30):
        jac, f0 = jacobian(f, x)
        if math.sqrt(np.dot(f0,f0)/len(x)) < tol:
            return x
        dx = MyFunc.gaussPivot(jac, -f0)
        x = x + dx
        if math.sqrt(np.dot(dx,dx)) < tol*max(max(abs(x)),1.0):
            return x
    print('Too many iterations')
    return x

In [49]:
def evaluatePoly(a,x):
    n = len(a) - 1
    p = a[n]
    dp = 0.0 + 0.0j
    ddp = 0.0 + 0.0j
    for i in range(1,n+1):
        ddp = ddp*x + 2.0*dp
        dp = dp*x + p
        p = p*x + a[n-i]
    return p,dp,ddp

In [51]:
import numpy as np
import cmath
from random import random
import MyFunc

def polyRoots(a,tol=1.0e-12):
    def laguerre(a,tol):
        x = random() # Starting value (random number)
        n = len(a) - 1
        for i in range(30):
            p,dp,ddp = MyFunc.evaluatePoly(a,x)
            if abs(p) < tol: return x
            g = dp/p
            h = g*g - ddp/p
            f = cmath.sqrt((n - 1)*(n*h - g*g))
            if abs(g + f) > abs(g - f): dx = n/(g + f)
            else: dx = n/(g - f)
            x = x - dx
            if abs(dx) < tol: return x
        print('Too many iterations')

    def deflPoly(a,root): # Deflates a polynomial
        n = len(a)-1
        b = [(0.0 + 0.0j)]*n
        b[n-1] = a[n]
        for i in range(n-2,-1,-1):
            b[i] = a[i+1] + root*b[i+1]
        return b

    n = len(a) - 1
    roots = np.zeros((n),dtype=complex)
    for i in range(n):
        x = laguerre(a,tol)
        if abs(x.imag) < tol: x = x.real
        roots[i] = x
        a = deflPoly(a,x)
    return roots

def deflPoly(a,root): # Deflates a polynomial
        n = len(a)-1
        b = [(0.0 + 0.0j)]*n
        b[n-1] = a[n]
        for i in range(n-2,-1,-1):
            b[i] = a[i+1] + root*b[i+1]
        return b

In [53]:
## module gaussNodes
'''x,A = gaussNodes(m,tol=10e-9)
Returns nodal abscissas {x} and weights {A} of
Gauss-Legendre m-point quadrature.
'''
import math
import numpy as np

def gaussNodes(m,tol=10e-9):

    def legendre(t,m):
        p0 = 1.0; p1 = t
        for k in range(1,m):
            p = ((2.0*k + 1.0)*t*p1 - k*p0)/(1.0 + k )
            p0 = p1; p1 = p
        dp = m*(p0 - t*p1)/(1.0 - t**2)
        return p,dp

    A = np.zeros(m)
    x = np.zeros(m)
    nRoots = int((m + 1)/2) # Number of non-neg. roots
    for i in range(nRoots):
        t = math.cos(math.pi*(i + 0.75)/(m + 0.5))# Approx. root
        for j in range(30):
            p,dp = legendre(t,m) # Newton-Raphson
            dt = -p/dp; t = t + dt # method
            if abs(dt) < tol:
                x[i] = t; x[m-i-1] = -t
                A[i] = 2.0/(1.0 - t**2)/(dp**2) # Eq.(6.25)
                A[m-i-1] = A[i]
                break
    return x,A

In [55]:
def gaussQuad(f,a,b,m):
    c1 = (b + a)/2.0
    c2 = (b - a)/2.0
    x,A = gaussNodes(m)
    sum = 0.0
    for i in range(len(x)):
        sum = sum + A[i]*f(c1 + c2*x[i])
    return c2*sum

In [57]:
## module gaussQuad2
''' I = gaussQuad2(f,xc,yc,m).
Gauss-Legendre integration of f(x,y) over a
quadrilateral using integration order m.
{xc},{yc} are the corner coordinates of the quadrilateral.
'''

import numpy as np

def gaussQuad2(f, x, y, m):

    def jac(x, y, s, t):
        J = np.zeros((2, 2))
        J[0, 0] = -(1.0 - t) * x[0] + (1.0 - t) * x[1] \
            + (1.0 + t) * x[2] - (1.0 + t) * x[3]
        J[0, 1] = -(1.0 - t) * y[0] + (1.0 - t) * y[1] \
            + (1.0 + t) * y[2] - (1.0 + t) * y[3]
        J[1, 0] = -(1.0 - s) * x[0] - (1.0 + s) * x[1] \
            + (1.0 + s) * x[2] + (1.0 - s) * x[3]
        J[1, 1] = -(1.0 - s) * y[0] - (1.0 + s) * y[1] \
            + (1.0 + s) * y[2] + (1.0 - s) * y[3]
        return (J[0, 0] * J[1, 1] - J[0, 1] * J[1, 0]) / 16.0

    def map(x, y, s, t):
        N = np.zeros(4)
        N[0] = (1.0 - s) * (1.0 - t) / 4.0
        N[1] = (1.0 + s) * (1.0 - t) / 4.0
        N[2] = (1.0 + s) * (1.0 + t) / 4.0
        N[3] = (1.0 - s) * (1.0 + t) / 4.0
        xCoord = np.dot(N, x)
        yCoord = np.dot(N, y)
        return xCoord, yCoord

    s, A = gaussNodes(m)
    summation = 0.0
    for i in range(m):
        for j in range(m):
            xCoord, yCoord = map(x, y, s[i], s[j])
            summation += A[i] * A[j] * jac(x, y, s[i], s[j]) * f(xCoord, yCoord)
    return summation